In [10]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from tqdm import tqdm  # Biblioteca para exibir a barra de progresso

# Carregar o arquivo CSV e extrair a coluna 'Comentario'
file_path = 'C:\\Users\\Caio\\Desktop\\TCC\\BASE_CIDADES.xlsx'
df = pd.read_excel(file_path)

In [11]:
# Verifique se a coluna 'Comentario' existe no arquivo
if 'Comentario' not in df.columns:
    raise ValueError("A coluna 'Comentario' não foi encontrada no arquivo.")

In [12]:
# Limpar dados, removendo NaNs e garantindo que todos os inputs sejam strings
df = df.dropna(subset=['Comentario'])
df['Comentario'] = df['Comentario'].astype(str)

# Converter a coluna 'Comentario' para uma lista de inputs
inputs = df['Comentario'].tolist()

# Pasta que contém os arquivos do modelo
model_path = r'C:\Users\Caio\Desktop\TCC\fine_tuned_model'

# Carregar o modelo e o tokenizer
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Criar o pipeline de classificação (usando top_k=None para obter todas as pontuações)
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, top_k=None, truncation=True, max_length=512 ) # Limita a 512 tokens

# Definir o threshold para filtragem de previsões
threshold = 0.3

In [ ]:
# Preparar listas para armazenar os resultados
labels = []
scores = []

# Definir o tamanho do lote (batch size)
batch_size = 20  # Você pode ajustar isso conforme necessário

# Iterar pelas entradas em lotes
for i in tqdm(range(0, len(inputs), batch_size), desc="Processando Comentários"):
    batch_inputs = inputs[i:i + batch_size]
    predictions = classifier(batch_inputs)

    for j, prediction in enumerate(predictions):
        best_label = None
        best_score = 0.0

        for label in prediction:
            if label['score'] >= threshold and label['score'] > best_score:
                best_label = label['label']
                best_score = label['score']

        labels.append(best_label)
        scores.append(best_score)

# Adicionar as novas colunas ao DataFrame original
df['Label'] = labels
df['Score'] = scores

# Defina o caminho para salvar o arquivo de saída como Excel
output_file = 'C:\\Users\\Caio\\Desktop\\TCC\\resultados.xlsx'

# Supondo que 'df' seja o DataFrame com os resultados que você quer salvar
df.to_excel(output_file, index=False)

print(f'Resultado salvo em: {output_file}')
